# Measure a Profile

## Note

All functions in this section using the cluster object as first argument are also internal functions of the cluster object, and should be used as such. They are just explicitely used here for clarity.

Ex:

```
theta, g_t, g_x = compute_shear(cl, geometry="flat")
```

should be done by the user as:

```
theta, g_t, g_x = cl.compute_shear(geometry="flat")
```

In [ ]:
import matplotlib.pyplot as plt
import clmm
from clmm.polaraveraging import compute_shear, make_shear_profile, make_bins
from clmm.plotting import plot_profiles
from clmm.galaxycluster import GalaxyCluster
import sys
sys.path.append('./support')
import mock_data as mock

Make sure we know which version we're using

In [ ]:
clmm.__version__

### Define cosmology object

In [ ]:
from astropy.cosmology import FlatLambdaCDM
mock_cosmo = FlatLambdaCDM(H0=70., Om0=0.3, Ob0=0.025)

### 1. Generate cluster object with mock data with shape noise, galaxies from redshift distribution and a pdz for each source galaxy

Define toy cluster parameters for mock data generation

In [ ]:
cosmo = mock_cosmo
cluster_id = "Awesome_cluster"
cluster_m = 1.e15
cluster_z = 0.3
concentration = 4
ngals = 1000
Delta = 200

zsrc_min = cluster_z + 0.1 # we only want to draw background galaxies

noisy_data_z = mock.generate_galaxy_catalog(cluster_m,
                                            cluster_z,
                                            concentration,
                                            cosmo,
                                            ngals,
                                            Delta,
                                            'chang13',
                                            zsrc_min = zsrc_min,
                                            shapenoise=0.005,
                                            photoz_sigma_unscaled=0.05)

Loading this into a CLMM cluster object centered on (0,0)

In [ ]:
cluster_ra = 0.0
cluster_dec = 0.0
gc_object = GalaxyCluster(cluster_id, cluster_ra, cluster_dec, 
                               cluster_z, noisy_data_z)

### 2. Load cluster object containing:
> Lens properties (ra_l, dec_l, z_l)

> Source properties (ra_s, dec_s, e1, e2)
### Note, if loading from mock data, use: 
>> cl = gc.load_cluster("GC_from_mock_data.pkl")

In [ ]:
cl = gc_object
print("Cluster info = ID:", cl.unique_id, "; ra:", cl.ra,
      "; dec:", cl.dec, "; z_l :", cl.z)
print("The number of source galaxies is :", len(cl.galcat))

Plot cluster and galaxy positions, and redshift distribution

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].scatter(cl.galcat['ra'], cl.galcat['dec'], color='blue', s=1, alpha=0.3)
ax[0].plot(cl.ra, cl.dec, 'ro')
ax[0].set_ylabel('dec', fontsize="large")
ax[0].set_xlabel('ra', fontsize="large")

hist = ax[1].hist(cl.galcat['z'], bins=40)[0]
ax[1].axvline(cl.z, c='r', ls='--')
ax[1].set_xlabel('$z_{source}$', fontsize="large")
xt = {t:f'{t}' for t in ax[1].get_xticks() if t!=0}
xt[cl.z] ='$z_{cl}$'
xto = sorted(list(xt.keys())+[cl.z])
ax[1].set_xticks(xto)
ax[1].set_xticklabels(xt[t] for t in xto)
ax[1].get_xticklabels()[xto.index(cl.z)].set_color('red')
plt.xlim(0, max(xto))
plt.show()

In [ ]:
min(cl.galcat['z']), cl.z

Check the ellipticities

In [ ]:
fig, ax1 = plt.subplots(1, 1)

ax1.scatter(cl.galcat['e1'], cl.galcat['e2'], s=1, alpha=0.2)
ax1.set_xlabel('e1')
ax1.set_ylabel('e2')
ax1.set_aspect('equal', 'datalim')
ax1.axvline(0, linestyle='dotted', color='black')
ax1.axhline(0, linestyle='dotted', color='black')

### 3. Compute and plot shear profiles

Compute angular separation, cross and tangential shear for each source galaxy

In [ ]:
theta, g_t, g_x = compute_shear(cl, geometry="flat")

Plot tangential and shear distributions for verification, which can be accessed in the galaxy cluster object, cl.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].hist(cl.galcat['gt'],bins=50)
ax[0].set_xlabel('$\\gamma_t$',fontsize='xx-large')

ax[1].hist(cl.galcat['gx'],bins=50)
ax[1].set_xlabel('$\\gamma_x$',fontsize='xx-large')
ax[1].set_yscale('log')

Compute transversal and cross shear profiles in units defined by user, using defaults binning 

In [ ]:
profiles = make_shear_profile(cl, "radians", "kpc", cosmo=cosmo)
profiles

Use function to plot the profiles

In [ ]:
fig, ax = plot_profiles(cl)

Shear Profile example in degrees

In [ ]:
new_profiles = make_shear_profile(cl, "radians", "degrees",
                                     cosmo=cosmo)
fig1, ax1 = plot_profiles(cl, "degrees")

With user defined binning, compute transversal and cross shear profiles in units defined by user, plot the new profiles

In [ ]:
new_bins = make_bins(1, 6, 20)

new_profiles = make_shear_profile(cl, "radians", "Mpc",
                                     bins=new_bins, cosmo=cosmo)

fig1, ax1 = plot_profiles(cl, "Mpc", r_units='Mpc')

You can also access the individual profile quantities 

In [ ]:
plt.title('cross shear test')
plt.errorbar(new_profiles['radius'], new_profiles['gx'],
             new_profiles['gx_err'])
plt.axhline(0, linestyle='dotted', color='black')
plt.xlabel("Radius [Mpc]")
plt.ylabel('$\\gamma_x$')
plt.show()

plt.title('Average redshift in radial bins')
plt.errorbar(new_profiles['radius'], new_profiles['z'],
             new_profiles['z_err'])
plt.axhline(cl.z, linestyle='dotted', color='r')
plt.text(1, cl.z*1.1, '$z_{cl}$', color='r')
plt.xlabel("Radius [Mpc]")
plt.ylabel('$\langle z\\rangle$')
plt.show()

### 4. Focus on some options
4.1. The `gal_ids_in_bins` option adds a `gal_id` field to the profile astropy table. For each bin of the profile, this is filled with the list of galaxy IDs for the galaxies that have fallen in that bin.

In [ ]:
profiles = make_shear_profile(cl, "radians", "kpc", cosmo=cosmo, gal_ids_in_bins=True)

In [ ]:
# Here the list of galaxy IDs that are in the first bin of the tangential shear profile
import numpy as np
gal_list = profiles['gal_id'][0]
print(gal_list)

In [ ]:
# We can then use that list to recover information of this subset of 
# galaxies in the original galaxy catalog

cl.galcat[gal_list]